Ejemplo tomado de https://www.maskaravivek.com/post/yolov1/
## Implementando YOLOv1 usando keras con tensorflow

En este notebook implementamos YOLOv1 como descripto originalmente en
[You Only Look Once](https://arxiv.org/abs/1506.02640). El objetivo es replicar el modelo descripto en el paper y en entender los detalles de usar keras para resolver un problema más complejo.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt 
%matplotlib inline

## Pre-procesamiento de datos

Vamos a usar [Pascal VOC 2007](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/) (Pascal Visual Object Classes) dado que el tamaño es más manejable y se puede correr en el colab.

Las clases

Primero, descargamos el dataset:

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar

!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar

!rm VOCtrainval_06-Nov-2007.tar
!rm VOCtest_06-Nov-2007.tar

El dataset `VOCtrainval_06-Nov-2007.tar` está organizado de la siguiente manera:

* VOC2007/JPEGImages/\<xxxxxx>.jpg: imágenes 
* VOC2007/ImageSets/Main/\<class>_\<imgset>.txt: especifica en cada imagen si está o no la clase correspondiente (para cada imageset = [train, val, trainval]
* VOC2007/ImageSets/Layout: \<imgset>.txt: archivos que determinan cada imagen a qué gruopo pertenece
* VOC2007/Annotations/\<xxxxxx>.xml: archivo xml que determina las bounding boxes de los objetos y sus clases en cada imagen, estructura: 

`<annotation> ... 
<size> 
<width>[width]</width>
<height>[height]</height>
<depth>[depth]</depth>
</size> ... `

`<object><name>[clase]</name> ... <bndbox><xmin>[xmin]</xmin><ymin>[ymin]</ymin>
<xmax>[xmin]</xmax><ymax>[ymin]</ymax></object>`



Las clases son:  

* Personas: person
* Animales: bird, cat, cow, dog, horse, sheep
* Vehiculos: aeroplane, bicycle, boat, bus, car, motorbike, train
* Objetos de interior: bottle, chair, dining table, potted plant, sofa, tv/monitor


20 clases en total. 


Ahora procesamos las annotations y obtenemos los labels ya que es más fácil consumir un archivo de texto en vez de XML.

In [ ]:
import argparse
import xml.etree.ElementTree as ET
import os

parser = argparse.ArgumentParser(description='Build Annotations.')
parser.add_argument('dir', default='..', help='Annotations.')

sets = [('2007', 'train'), ('2007', 'val'), ('2007', 'test')]

classes_num = {'aeroplane': 0, 'bicycle': 1, 'bird': 2, 'boat': 3, 'bottle': 4, 'bus': 5,
               'car': 6, 'cat': 7, 'chair': 8, 'cow': 9, 'diningtable': 10, 'dog': 11,
               'horse': 12, 'motorbike': 13, 'person': 14, 'pottedplant': 15, 'sheep': 16,
               'sofa': 17, 'train': 18, 'tvmonitor': 19}


def convert_annotation(year, image_id, f):
  '''
     input: 
        year: año de dataset VOC
        image_id: id de la imagen
        f: archivo donde escribir la salidad
  '''
    in_file = os.path.join('VOCdevkit/VOC%s/Annotations/%s.xml' % (year, image_id))
    tree = ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        # por cada objeto en el xml de anotacion de la imagen
        difficult = obj.find('difficult').text
        cls = obj.find('name').text # encontrar la clase
        classes = list(classes_num.keys())
        if cls not in classes or int(difficult) == 1: # descartar objetos invalidos
            continue
        cls_id = classes.index(cls) # obtener indice de la clase del objeto
        xmlbox = obj.find('bndbox')
        # obtener las dimensiones de la bounding box del objeto
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('ymin').text), 
             int(xmlbox.find('xmax').text), int(xmlbox.find('ymax').text))
        # escribir dimensiones e indice de la clase del objeto en la linea 
        f.write(' ' + ','.join([str(a) for a in b]) + ',' + str(cls_id))

In [ ]:
for year, image_set in sets:
  # image_set: set de imagenes que interesa train, val, test
  print(year, image_set)
  with open(os.path.join('VOCdevkit/VOC%s/ImageSets/Main/%s.txt' % (year, image_set)), 'r') as f:
    # obtener la lista de imagenes en el set
      image_ids = f.read().strip().split()
  with open(os.path.join("VOCdevkit", '%s_%s.txt' % (year, image_set)), 'w') as f:
      # por aca set hacer output de las imagenes 
      for image_id in image_ids:
          f.write('%s/VOC%s/JPEGImages/%s.jpg' % ("VOCdevkit", year, image_id)) # escribir
          convert_annotation(year, image_id, f) # usar la funcion anterior 
          f.write('\n')

2007 train
2007 val
2007 test


Ahora hacemos una función que prepara la entrada y la salida. La entrada son imágenes de (448, 448, 3) y la salida son tensores de dimensión (7, 7, 30) = (S, S, \[B\*(1 + 4) +C\]), donde S\*S es el tamaño de la grilla con S= 7. B = 2 es la cantidad de bounding boxes por cada celda de la grilla, C = 20 es el número de categorías.

In [ ]:
import cv2 as cv
import numpy as np

def read(image_path, label):
    image = cv.imread(image_path)
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    image_h, image_w = image.shape[0:2]
    image = cv.resize(image, (448, 448)) # re-escalamos la imagen para que mida 448x448
    image = image / 255. # se normaliza el valor de los pixels entre 0 y 1

    label_matrix = np.zeros([7, 7, 30])
    for l in label:
        l = l.split(',')
        l = np.array(l, dtype=np.int)
        xmin = l[0]
        ymin = l[1]
        xmax = l[2]
        ymax = l[3]
        cls = l[4]
        x = (xmin + xmax) / 2 / image_w # obtenemos el centro en x del objeto como proporcion de la imagen total
        y = (ymin + ymax) / 2 / image_h # obtenemos el centro en y del objeto como proporcion de la imagen total
        w = (xmax - xmin) / image_w # obtenemos la dimension x del objeto como proporcion de la imagen total
        h = (ymax - ymin) / image_h # obtenemos la dimension y del objeto como proporcion de la imagen total
        loc = [7 * x, 7 * y]
        loc_i = int(loc[1]) # en qué celda de la grilla 7x7 se encuentra el objecto (dimension y)
        loc_j = int(loc[0]) # en qué celda de la grilla 7x7 se encuentra el objecto (dimension x)
        y = loc[1] - loc_i # posicion del centro con respecto a la celda (dimension y) 
        x = loc[0] - loc_j # posicion del centro con respecto a la celda (dimension x)

        if label_matrix[loc_i, loc_j, 24] == 0:
            label_matrix[loc_i, loc_j, cls] = 1 # clase a la que pertence el objeto (one hot encoding)
            label_matrix[loc_i, loc_j, 20:24] = [x, y, w, h] # posicion del objeto en la celda
            label_matrix[loc_i, loc_j, 24] = 1  # objeto presente

    return image, label_matrix

## Entrenando el modelo

Definimos un custom generator que retorna un batch de entradas y salidas.


In [ ]:
from tensorflow import keras

# NOTA: podría extenderse el generador para realizar data augmentation, en el paper
# original se introduce re escalado aleatorio y traslaciones de hasta 20% de la 
# tamaño de la imagen original.
# también se distorsiona la imagen en su exposición y saturación de colores hasta
# un factor de 1.5 en espacio de colores HSV
class My_Custom_Generator(keras.utils.Sequence):
  
  def __init__(self, images, labels, batch_size):
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.images[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

    train_image = []
    train_label = []

    for i in range(0, len(batch_x)):
      img_path = batch_x[i]
      label = batch_y[i]
      image, label_matrix = read(img_path, label)
      train_image.append(image)
      train_label.append(label_matrix)
    return np.array(train_image), np.array(train_label)


Preparamos arrays con entradas y salidas para los sets de entrenamiento y validación.

In [ ]:
train_datasets = []
val_datasets = []

with open(os.path.join("VOCdevkit", '2007_train.txt'), 'r') as f:
    train_datasets = train_datasets + f.readlines()
with open(os.path.join("VOCdevkit", '2007_val.txt'), 'r') as f:
    val_datasets = val_datasets + f.readlines()

X_train = []
Y_train = []

X_val = []
Y_val = []

for item in train_datasets:
  item = item.replace("\n", "").split(" ")
  X_train.append(item[0])
  arr = []
  for i in range(1, len(item)):
    arr.append(item[i])
  Y_train.append(arr)

for item in val_datasets:
  item = item.replace("\n", "").split(" ")
  X_val.append(item[0])
  arr = []
  for i in range(1, len(item)):
    arr.append(item[i])
  Y_val.append(arr)

Creamos instancias del generador para entrenamiento y validación.

In [ ]:
batch_size = 4
my_training_batch_generator = My_Custom_Generator(X_train, Y_train, batch_size)

my_validation_batch_generator = My_Custom_Generator(X_val, Y_val, batch_size)

x_train, y_train = my_training_batch_generator.__getitem__(0)
x_val, y_val = my_training_batch_generator.__getitem__(0)
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(4, 448, 448, 3)
(4, 7, 7, 30)
(4, 448, 448, 3)
(4, 7, 7, 30)


### Definir una capa de salida customizada

Necesitamos hacer reshape de la salida del modelo así que definimos una capa de Keras customizada.


In [ ]:
from tensorflow import keras
import keras.backend as K

class Yolo_Reshape(tf.keras.layers.Layer):
  def __init__(self, target_shape):
    super(Yolo_Reshape, self).__init__()
    self.target_shape = tuple(target_shape)

  def get_config(self):
    config = super().get_config().copy()
    config.update({
        'target_shape': self.target_shape
    })
    return config

  def call(self, input):
    # grilla de 7x7
    S = [self.target_shape[0], self.target_shape[1]]
    # cantidad de clases
    C = 20
    # cantidad de bounding boxes por celda
    B = 2

    idx1 = S[0] * S[1] * C
    idx2 = idx1 + S[0] * S[1] * B
    
    # probabilidades de cada clase
    class_probs = K.reshape(input[:, :idx1], (K.shape(input)[0],) + tuple([S[0], S[1], C]))
    class_probs = K.softmax(class_probs)

    # confianza de cada clase
    confs = K.reshape(input[:, idx1:idx2], (K.shape(input)[0],) + tuple([S[0], S[1], B]))
    confs = K.sigmoid(confs)

    # bounding boxes
    boxes = K.reshape(input[:, idx2:], (K.shape(input)[0],) + tuple([S[0], S[1], B * 4]))
    boxes = K.sigmoid(boxes)

    outputs = K.concatenate([class_probs, confs, boxes])
    return outputs

### Definimos el modelo YOLO

Lo definimos según el paper original.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, Flatten, Reshape, LeakyReLU
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2

lrelu = LeakyReLU(alpha=0.1)

nb_boxes=1
grid_w=7
grid_h=7
cell_w=64
cell_h=64
img_w=grid_w*cell_w
img_h=grid_h*cell_h

model = Sequential()
model.add(Conv2D(filters=64, kernel_size= (7, 7), strides=(1, 1), input_shape =(img_h, img_w, 3), padding = 'same', activation=lrelu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'same'))

model.add(Conv2D(filters=192, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'same'))

model.add(Conv2D(filters=128, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=256, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'same'))

model.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=256, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding = 'same'))

model.add(Conv2D(filters=512, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=512, kernel_size= (1, 1), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=1024, kernel_size= (3, 3), padding = 'same', activation=lrelu))
model.add(Conv2D(filters=1024, kernel_size= (3, 3), strides=(2, 2), padding = 'same'))

model.add(Conv2D(filters=1024, kernel_size= (3, 3), activation=lrelu))
model.add(Conv2D(filters=1024, kernel_size= (3, 3), activation=lrelu))

model.add(Flatten())
model.add(Dense(512))
model.add(Dense(1024))
model.add(Dropout(0.5))
model.add(Dense(1470, activation='sigmoid'))
model.add(Yolo_Reshape(target_shape=(7,7,30))) # hacemos reshape de la salida
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 448, 448, 64)      9472      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 192)     110784    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 112, 112, 192)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     24704     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 256)     295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 112, 112, 256)     6

### Definimos una evolución del learning rate

El paper utiliza learning rates diferentes por cada epoch. Así que definimos una función de callback para implementarlo.


In [ ]:
from tensorflow import keras

class CustomLearningRateScheduler(keras.callbacks.Callback):
    def __init__(self, schedule):
        super(CustomLearningRateScheduler, self).__init__()
        self.schedule = schedule

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('El optimizador debe tener un atributo "lr".')
        # Obtener el learning rate actual del optimizer
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        # llamar a la función que determina el learning rate que debe valer ahorra
        scheduled_lr = self.schedule(epoch, lr)
        # setear el valor que debe valer en el optimizer antes de que empiece esta epoch
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nEpoch %05d: Learning rate = %6.4f." % (epoch, scheduled_lr))


LR_SCHEDULE = [
    # tuplas de (epoch de comienzo, learning rate) 
    (0, 0.01),
    (75, 0.001),
    (105, 0.0001),
]

def lr_schedule(epoch, lr):
    """ Función helper para recuperar el learning rate según el schedule"""
    if epoch < LR_SCHEDULE[0][0] or epoch > LR_SCHEDULE[-1][0]:
        return lr
    for i in range(len(LR_SCHEDULE)):
        if epoch == LR_SCHEDULE[i][0]:
            return LR_SCHEDULE[i][1]
    return lr

### Definimos la función de pérdida

Ahora definimos una función de pérdida customizada para usar en el modelo. Para entender más acerca de la función de pérdida leer el post [Understanding YOLO](https://hackernoon.com/understanding-yolo-f5a74bbc7967). 

La implementación de la función de pérdida fue tomada de este [repo de Github](https://github.com/JY-112553/yolov1-keras-voc).

In [ ]:
import keras.backend as K


def xywh2minmax(xy, wh):
    xy_min = xy - wh / 2
    xy_max = xy + wh / 2

    return xy_min, xy_max


def iou(pred_mins, pred_maxes, true_mins, true_maxes):
    intersect_mins = K.maximum(pred_mins, true_mins)
    intersect_maxes = K.minimum(pred_maxes, true_maxes)
    intersect_wh = K.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]

    pred_wh = pred_maxes - pred_mins
    true_wh = true_maxes - true_mins
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]
    true_areas = true_wh[..., 0] * true_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores = intersect_areas / union_areas

    return iou_scores


def yolo_head(feats):
    # implementación dinámica de dimensiones de convolución para un modelo convolucional
    conv_dims = K.shape(feats)[1:3]  # asumimos los canales en la última dimensión
    # En YOLO el índice de la altura es la iteración es la iteración central
    conv_height_index = K.arange(0, stop=conv_dims[0])
    conv_width_index = K.arange(0, stop=conv_dims[1])
    conv_height_index = K.tile(conv_height_index, [conv_dims[1]])

    # conv_width_index = K.repeat_elements(conv_width_index, conv_dims[1], axis=0)
    conv_width_index = K.tile(
        K.expand_dims(conv_width_index, 0), [conv_dims[0], 1])
    conv_width_index = K.flatten(K.transpose(conv_width_index))
    conv_index = K.transpose(K.stack([conv_height_index, conv_width_index]))
    conv_index = K.reshape(conv_index, [1, conv_dims[0], conv_dims[1], 1, 2])
    conv_index = K.cast(conv_index, K.dtype(feats))

    conv_dims = K.cast(K.reshape(conv_dims, [1, 1, 1, 1, 2]), K.dtype(feats))

    box_xy = (feats[..., :2] + conv_index) / conv_dims * 448
    box_wh = feats[..., 2:4] * 448

    return box_xy, box_wh


def yolo_loss(y_true, y_pred):
    label_class = y_true[..., :20]  # ? * 7 * 7 * 20
    label_box = y_true[..., 20:24]  # ? * 7 * 7 * 4
    response_mask = y_true[..., 24]  # ? * 7 * 7
    response_mask = K.expand_dims(response_mask)  # ? * 7 * 7 * 1

    predict_class = y_pred[..., :20]  # ? * 7 * 7 * 20
    predict_trust = y_pred[..., 20:22]  # ? * 7 * 7 * 2
    predict_box = y_pred[..., 22:]  # ? * 7 * 7 * 8

    _label_box = K.reshape(label_box, [-1, 7, 7, 1, 4])
    _predict_box = K.reshape(predict_box, [-1, 7, 7, 2, 4])

    label_xy, label_wh = yolo_head(_label_box)  # ? * 7 * 7 * 1 * 2, ? * 7 * 7 * 1 * 2
    label_xy = K.expand_dims(label_xy, 3)  # ? * 7 * 7 * 1 * 1 * 2
    label_wh = K.expand_dims(label_wh, 3)  # ? * 7 * 7 * 1 * 1 * 2
    label_xy_min, label_xy_max = xywh2minmax(label_xy, label_wh)  # ? * 7 * 7 * 1 * 1 * 2, ? * 7 * 7 * 1 * 1 * 2

    predict_xy, predict_wh = yolo_head(_predict_box)  # ? * 7 * 7 * 2 * 2, ? * 7 * 7 * 2 * 2
    predict_xy = K.expand_dims(predict_xy, 4)  # ? * 7 * 7 * 2 * 1 * 2
    predict_wh = K.expand_dims(predict_wh, 4)  # ? * 7 * 7 * 2 * 1 * 2
    predict_xy_min, predict_xy_max = xywh2minmax(predict_xy, predict_wh)  # ? * 7 * 7 * 2 * 1 * 2, ? * 7 * 7 * 2 * 1 * 2

    iou_scores = iou(predict_xy_min, predict_xy_max, label_xy_min, label_xy_max)  # ? * 7 * 7 * 2 * 1
    best_ious = K.max(iou_scores, axis=4)  # ? * 7 * 7 * 2
    best_box = K.max(best_ious, axis=3, keepdims=True)  # ? * 7 * 7 * 1

    box_mask = K.cast(best_ious >= best_box, K.dtype(best_ious))  # ? * 7 * 7 * 2

    no_object_loss = 0.5 * (1 - box_mask * response_mask) * K.square(0 - predict_trust)
    object_loss = box_mask * response_mask * K.square(1 - predict_trust)
    confidence_loss = no_object_loss + object_loss
    confidence_loss = K.sum(confidence_loss)

    class_loss = response_mask * K.square(label_class - predict_class)
    class_loss = K.sum(class_loss)

    _label_box = K.reshape(label_box, [-1, 7, 7, 1, 4])
    _predict_box = K.reshape(predict_box, [-1, 7, 7, 2, 4])

    label_xy, label_wh = yolo_head(_label_box)  # ? * 7 * 7 * 1 * 2, ? * 7 * 7 * 1 * 2
    predict_xy, predict_wh = yolo_head(_predict_box)  # ? * 7 * 7 * 2 * 2, ? * 7 * 7 * 2 * 2

    box_mask = K.expand_dims(box_mask)
    response_mask = K.expand_dims(response_mask)

    box_loss = 5 * box_mask * response_mask * K.square((label_xy - predict_xy) / 448)
    box_loss += 5 * box_mask * response_mask * K.square((K.sqrt(label_wh) - K.sqrt(predict_wh)) / 448)
    box_loss = K.sum(box_loss)

    loss = confidence_loss + class_loss + box_loss

    return loss

### Agregar el callback para guardar los pesos

In [ ]:
# funciona para guardar los pesos del mejor modelo
from tensorflow.keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

### Compilar el modelo con función de pérdidad de más arriba


In [ ]:
from tensorflow import keras

model.compile(loss=yolo_loss ,optimizer='adam')

### Entrenar el modelo 

Entrenamos durante 135 epochs con `model.fit`

In [ ]:
model.fit(x=my_training_batch_generator,
          steps_per_epoch = int(len(X_train) // batch_size),
          epochs = 135,
          verbose = 1,
          workers= 4,
          validation_data = my_validation_batch_generator,
          validation_steps = int(len(X_val) // batch_size),
           callbacks=[
              CustomLearningRateScheduler(lr_schedule),
              mcp_save
          ])


Epoch 00000: Learning rate is 0.0100.
Epoch 1/135
625/625 [==============================] - 194s 310ms/step - loss: 63.7584 - val_loss: 59.6588

Epoch 00001: Learning rate is 0.0100.
Epoch 2/135
625/625 [==============================] - 188s 301ms/step - loss: 63.6997 - val_loss: 59.6588

Epoch 00002: Learning rate is 0.0100.
Epoch 3/135
625/625 [==============================] - 188s 301ms/step - loss: 63.7505 - val_loss: 59.6588

Epoch 00003: Learning rate is 0.0100.
Epoch 4/135
625/625 [==============================] - 188s 301ms/step - loss: 63.8414 - val_loss: 59.6588

Epoch 00004: Learning rate is 0.0100.
Epoch 5/135
625/625 [==============================] - 188s 301ms/step - loss: 63.7653 - val_loss: 59.6588

Epoch 00005: Learning rate is 0.0100.
Epoch 6/135
625/625 [==============================] - 188s 301ms/step - loss: 63.9053 - val_loss: 59.6588

Epoch 00006: Learning rate is 0.0100.
Epoch 7/135
625/625 [==============================] - 188s 301ms/step - loss: 63.756

## Conclusión

Esta implementación construye YOLO V1 de primeros principios. La implementación no obtiene los mismos resultados que el paper original porque falta el paso de pretraining con imágenes de imagenet (seccion 2.2 del paper de YOLO). En el paper se pre entrenan las 20 primeras capas convolucionales seguidas de una capa de Average Pooling y una capa Fully Connected. Se pre entrenó esa red por una semana hasta llegar a top-5 accuracy de 88% en el set de validación de Imagenet 2012. Luego se convierte la red para realizar detección, se agregan cuatro capas convolucionales y 2 fully connected con pesos aleatorios. Los pesos 